# Embryo Data Query

This notebook queries embryo-related data from multiple databases for a given prontuario/PIN.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_congelamentos_embrioes
  - view_descongelamentos_embrioes
  - view_embrioes_congelados
- **Huntington Data Lake** (huntington_data_lake.duckdb):
  - gold.clinisys_embrioes
  - gold.embryoscope_embrioes
  - gold.embryoscope_clinisys_combined
  - gold.embryoscope_treatment (from embryoscope_embrioes)
  - silver.planilha_embriologia
  - gold.planilha_embryoscope_combined


In [57]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Database Connections


In [58]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Query Functions - Clinisys Tables


In [59]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [60]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    return df

def query_descongelamentos_embrioes(conn, prontuario):
    """Query view_descongelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_embrioes")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    return df

def query_congelamentos_ovulos(conn, prontuario):
    """Query view_congelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_ovulos")
    return df

def query_descongelamentos_ovulos(conn, prontuario):
    """Query view_descongelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_ovulos")
    return df

def query_ovulos_congelados(conn, prontuario):
    """Query view_ovulos_congelados for a specific prontuario"""
    print(f"\n=== VIEW_OVULOS_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_ovulos_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_ovulos_congelados")
    return df


## Query Functions - Huntington Data Lake Tables


In [61]:
def query_clinisys_embrioes(conn, prontuario):
    """Query gold.clinisys_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.CLINISYS_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.clinisys_embrioes 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.clinisys_embrioes")
    return df

def query_embryoscope_embrioes(conn, prontuario):
    """Query gold.embryoscope_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY embryo_KIDDate DESC, treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_embrioes")
    return df

def query_embryoscope_clinisys_combined(conn, prontuario):
    """Query gold.embryoscope_clinisys_combined for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINED ===")
    query = f"""
    SELECT * FROM gold.embryoscope_clinisys_combined 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_clinisys_combined")
    return df


In [62]:
def query_planilha_embriologia(conn, prontuario):
    """Query silver.planilha_embriologia for a specific prontuario"""
    print(f"\n=== SILVER.PLANILHA_EMBRIOLOGIA ===")
    query = f"""
    SELECT * FROM silver.planilha_embriologia 
    WHERE PIN = {prontuario}
    ORDER BY "DATA DA FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.planilha_embriologia")
    return df

def query_planilha_embryoscope_combined(conn, prontuario):
    """Query gold.planilha_embryoscope_combined for a specific prontuario"""
    print(f"\n=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===")
    query = f"""
    SELECT * FROM gold.planilha_embryoscope_combined 
    WHERE "micro_prontuario" = {prontuario} OR "planilha_PIN" = {prontuario}
    ORDER BY "descong_em_DataTransferencia" DESC, "planilha_DATA_DA_FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.planilha_embryoscope_combined")
    return df


## Main Query Function


In [63]:
def query_embryo_data(prontuario):
    """
    Query all embryo-related data for a given prontuario/PIN.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results
    """
    print(f"\n{'='*80}")
    print(f"QUERYING EMBRYO DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_descongelamentos_embrioes = query_descongelamentos_embrioes(clinisys_con, prontuario)
        df_embrioes_congelados = query_embrioes_congelados(clinisys_con, prontuario)
        df_congelamentos_ovulos = query_congelamentos_ovulos(clinisys_con, prontuario)
        df_descongelamentos_ovulos = query_descongelamentos_ovulos(clinisys_con, prontuario)
        df_ovulos_congelados = query_ovulos_congelados(clinisys_con, prontuario)
        
        # Query huntington data lake tables
        df_clinisys_embrioes = query_clinisys_embrioes(huntington_con, prontuario)
        df_embryoscope_embrioes = query_embryoscope_embrioes(huntington_con, prontuario)
        df_embryoscope_clinisys_combined = query_embryoscope_clinisys_combined(huntington_con, prontuario)
        # df_embryoscope_treatment = query_embryoscope_treatment(huntington_con, prontuario)
        df_planilha_embriologia = query_planilha_embriologia(huntington_con, prontuario)
        df_planilha_embryoscope_combined = query_planilha_embryoscope_combined(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_descongelamentos_embrioes: {len(df_descongelamentos_embrioes)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes_congelados)} records")
        print(f"- view_congelamentos_ovulos: {len(df_congelamentos_ovulos)} records")
        print(f"- view_descongelamentos_ovulos: {len(df_descongelamentos_ovulos)} records")
        print(f"- view_ovulos_congelados: {len(df_ovulos_congelados)} records")
        print(f"- gold.clinisys_embrioes: {len(df_clinisys_embrioes)} records")
        print(f"- gold.embryoscope_embrioes: {len(df_embryoscope_embrioes)} records")
        print(f"- gold.embryoscope_clinisys_combined: {len(df_embryoscope_clinisys_combined)} records")
        print(f"- silver.planilha_embriologia: {len(df_planilha_embriologia)} records")
        print(f"- gold.planilha_embryoscope_combined: {len(df_planilha_embryoscope_combined)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'descongelamentos_embrioes': df_descongelamentos_embrioes,
            'embrioes_congelados': df_embrioes_congelados,
            'congelamentos_ovulos': df_congelamentos_ovulos,
            'descongelamentos_ovulos': df_descongelamentos_ovulos,
            'ovulos_congelados': df_ovulos_congelados,
            'clinisys_embrioes': df_clinisys_embrioes,
            'embryoscope_embrioes': df_embryoscope_embrioes,
            'embryoscope_clinisys_combined': df_embryoscope_clinisys_combined,
            'planilha_embriologia': df_planilha_embriologia,
            'planilha_embryoscope_combined': df_planilha_embryoscope_combined
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Example Usage


In [209]:
# Example usage
prontuario = 138475 
prontuario = 809722
prontuario = 107805
prontuario = 510693; prontuario = 902572
prontuario = 65591; prontuario = 162377
prontuario = 751632
prontuario = 784771
# Query all embryo data
results = query_embryo_data(prontuario)



QUERYING EMBRYO DATA FOR PRONTUARIO: 784771
Query executed at: 2025-12-17 13:21:36.810508

=== VIEW_MICROMANIPULACAO ===
Found 0 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
No codigo_ficha found for this prontuario in view_micromanipulacao.

=== VIEW_TRATAMENTOS ===
Found 0 records in view_tratamentos

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 0 records in view_congelamentos_embrioes

=== VIEW_DESCONGELAMENTOS_EMBRIOES ===
Found 0 records in view_descongelamentos_embrioes

=== VIEW_EMBRIOES_CONGELADOS ===
Found 0 records in view_embrioes_congelados

=== VIEW_CONGELAMENTOS_OVULOS ===
Found 0 records in view_congelamentos_ovulos

=== VIEW_DESCONGELAMENTOS_OVULOS ===
Found 0 records in view_descongelamentos_ovulos

=== VIEW_OVULOS_CONGELADOS ===
Found 0 records in view_ovulos_congelados

=== GOLD.CLINISYS_EMBRIOES ===
Found 0 records in gold.clinisys_embrioes

=== GOLD.EMBRYOSCOPE_EMBRIOES ===
Found 0 records in gold.embryoscope_embrioes

=== GOLD.EMBRYOSCOPE_

## Display Results

### 1. Micromanipulacao


In [210]:
print("=== MICROMANIPULACAO DATA ===")
micromanipulacao_data = results['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulacao data found.")


=== MICROMANIPULACAO DATA ===
Records: 0
No micromanipulacao data found.


### 2. Micromanipulacao Oocitos


In [211]:
print("=== MICROMANIPULACAO OOCITOS DATA ===")
oocitos_data = results['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulacao oocitos data found.")


=== MICROMANIPULACAO OOCITOS DATA ===
Records: 0
No micromanipulacao oocitos data found.


### 3. Tratamentos


In [212]:
print("=== TRATAMENTOS DATA ===")
tratamentos_data = results['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No tratamentos data found.")


=== TRATAMENTOS DATA ===
Records: 0
No tratamentos data found.


### 4. Congelamentos Embrioes


In [213]:
print("=== CONGELAMENTOS EMBRIOES DATA ===")
congelamentos_data = results['congelamentos_embrioes']
print(f"Records: {len(congelamentos_data)}")
if not congelamentos_data.empty:
    display(congelamentos_data)
else:
    print("No congelamentos embrioes data found.")


=== CONGELAMENTOS EMBRIOES DATA ===
Records: 0
No congelamentos embrioes data found.


### 5. Descongelamentos Embrioes


In [214]:
print("=== DESCONGELAMENTOS EMBRIOES DATA ===")
descongelamentos_data = results['descongelamentos_embrioes']
print(f"Records: {len(descongelamentos_data)}")
if not descongelamentos_data.empty:
    display(descongelamentos_data)
else:
    print("No descongelamentos embrioes data found.")


=== DESCONGELAMENTOS EMBRIOES DATA ===
Records: 0
No descongelamentos embrioes data found.


### 6. Embrioes Congelados


In [215]:
print("=== EMBRIOES CONGELADOS DATA ===")
embrioes_congelados_data = results['embrioes_congelados']
print(f"Records: {len(embrioes_congelados_data)}")
if not embrioes_congelados_data.empty:
    # display(embrioes_congelados_data)
    display(embrioes_congelados_data[embrioes_congelados_data['id_descongelamento']!=0])
else:
    print("No embrioes congelados data found.")


=== EMBRIOES CONGELADOS DATA ===
Records: 0
No embrioes congelados data found.


### 7. Clinisys Embrioes


In [216]:
print("=== CLINISYS EMBRIOES DATA ===")
clinisys_embrioes_data = results['clinisys_embrioes']
print(f"Records: {len(clinisys_embrioes_data)}")
if not clinisys_embrioes_data.empty:
    display(clinisys_embrioes_data.head(10))
else:
    print("No clinisys embrioes data found.")


=== CLINISYS EMBRIOES DATA ===
Records: 0
No clinisys embrioes data found.


In [217]:
clinisys_embrioes_data[clinisys_embrioes_data['oocito_id'].isin([184181])]

,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [218]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'micro_codigo_ficha', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento'
]

clinisys_embrioes_data.loc[clinisys_embrioes_data['oocito_embryo_number'].isin([33,35]), colunas]

,oocito_id,oocito_id_micromanipulacao,oocito_embryo_number,oocito_TCD,micro_codigo_ficha,micro_numero_caso,micro_Data_DL,micro_data_procedimento,cong_em_id,cong_em_CodCongelamento,cong_em_Ciclo,cong_em_NEmbrioes,emb_cong_id,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,emb_cong_embriao,descong_em_id,descong_em_transferidos_transferencia,descong_em_CodDescongelamento,descong_em_Ciclo,descong_em_DataDescongelamento,descong_em_DataTransferencia,trat_id,trat_data_procedimento,trat_data_transferencia,trat_resultado_tratamento


### 8. Embryoscope Embrioes


In [219]:
print("=== EMBRYOSCOPE EMBRIOES DATA ===")
embryoscope_embrioes_data = results['embryoscope_embrioes']
print(f"Records: {len(embryoscope_embrioes_data)}")
if not embryoscope_embrioes_data.empty:
    display(embryoscope_embrioes_data)
else:
    print("No embryoscope embrioes data found.")


=== EMBRYOSCOPE EMBRIOES DATA ===
Records: 0
No embryoscope embrioes data found.


In [220]:
embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')

C:\Users\FilipeFurlanBellotti\AppData\Local\Temp\ipykernel_4300\2666312698.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_DateOfBirth,prontuario,patient_unit_huntington,treatment_PatientIDx,treatment_TreatmentName,treatment_unit_huntington,embryo_EmbryoID,embryo_KIDDate,embryo_KIDScore,embryo_KIDUser,embryo_KIDVersion,embryo_Description,embryo_EmbryoDescriptionID,embryo_EmbryoFate,embryo_FertilizationMethod,embryo_FertilizationTime,embryo_InstrumentNumber,embryo_Position,embryo_WellNumber,embryo_embryo_number,embryo_unit_huntington,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Value_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Timestamp_EVEN2,embryo_Value_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Timestamp_EVEN4,embryo_Value_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Timestamp_EVEN8,embryo_Value_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Timestamp_FRAG2,embryo_Value_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Timestamp_FRAG4,embryo_Value_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Timestamp_FRAG8,embryo_Value_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Value_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Timestamp_ICM,embryo_Value_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Timestamp_Line,embryo_Value_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Timestamp_MN2Type,embryo_Value_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Value_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Value_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Timestamp_PN,embryo_Value_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Timestamp_Pulsing,embryo_Value_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Timestamp_Strings,embryo_Value_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Timestamp_TE,embryo_Value_TE,embryo_Name_t2,embryo_Time_t2,embryo_Timestamp_t2,embryo_Value_t2,embryo_Name_t3,embryo_Time_t3,embryo_Timestamp_t3,embryo_Value_t3,embryo_Name_t4,embryo_Time_t4,embryo_Timestamp_t4,embryo_Value_t4,embryo_Name_t5,embryo_Time_t5,embryo_Timestamp_t5,embryo_Value_t5,embryo_Name_t6,embryo_Time_t6,embryo_Timestamp_t6,embryo_Value_t6,embryo_Name_t7,embryo_Time_t7,embryo_Timestamp_t7,embryo_Value_t7,embryo_Name_t8,embryo_Time_t8,embryo_Timestamp_t8,embryo_Value_t8,embryo_Name_t9,embryo_Time_t9,embryo_Timestamp_t9,embryo_Value_t9,embryo_Name_tB,embryo_Time_tB,embryo_Timestamp_tB,embryo_Value_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Timestamp_tEB,embryo_Value_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Timestamp_tHB,embryo_Value_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Timestamp_tM,embryo_Value_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Timestamp_tPB2,embryo_Value_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Timestamp_tPNa,embryo_Value_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Timestamp_tPNf,embryo_Value_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Timestamp_tSB,embryo_Value_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Timestamp_tSC,embryo_Value_tSC,embryo_PatientIDx,embryo_TreatmentName


### 9. Embryoscope Clinisys Combined


In [221]:
print("=== EMBRYOSCOPE CLINISYS COMBINED DATA ===")
combined_data = results['embryoscope_clinisys_combined']
print(f"Records: {len(combined_data)}")
if not combined_data.empty:
    display(combined_data.head(10))
else:
    print("No embryoscope clinisys combined data found.")


=== EMBRYOSCOPE CLINISYS COMBINED DATA ===
Records: 0
No embryoscope clinisys combined data found.


In [222]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'micro_codigo_ficha', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'treatment_TreatmentName', 'embryo_EmbryoID', 'embryo_KIDScore', 'embryo_Description', 'embryo_EmbryoDescriptionID', 'embryo_embryo_number',
    'emb_cong_transferidos', 'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento', 'trat_tentativa'
]

# combined_data.loc[combined_data['oocito_id_micromanipulacao'].isin([21961]), colunas]
combined_data.loc[combined_data['embryo_EmbryoID'].isin(['D2024.01.15_S03386_I3166_P-3', 'D2024.01.15_S03386_I3166_P-5']), colunas] 
# combined_data.loc[:, colunas]

,oocito_id,oocito_id_micromanipulacao,oocito_embryo_number,oocito_TCD,micro_codigo_ficha,micro_numero_caso,micro_Data_DL,micro_data_procedimento,cong_em_id,cong_em_CodCongelamento,cong_em_Ciclo,cong_em_NEmbrioes,treatment_TreatmentName,embryo_EmbryoID,embryo_KIDScore,embryo_Description,embryo_EmbryoDescriptionID,embryo_embryo_number,emb_cong_transferidos,emb_cong_id,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,emb_cong_embriao,descong_em_id,descong_em_transferidos_transferencia,descong_em_CodDescongelamento,descong_em_Ciclo,descong_em_DataDescongelamento,descong_em_DataTransferencia,trat_id,trat_data_procedimento,trat_data_transferencia,trat_resultado_tratamento,trat_tentativa


### 11. Planilha Embriologia


In [223]:
print("=== PLANILHA EMBRIOLOGIA DATA ===")
planilha_data = results['planilha_embriologia']
print(f"Records: {len(planilha_data)}")
if not planilha_data.empty:
    display(planilha_data)
else:
    print("No planilha embriologia data found.")


=== PLANILHA EMBRIOLOGIA DATA ===
Records: 0
No planilha embriologia data found.


### 12. Congelamentos Ovulos


In [224]:
print("=== CONGELAMENTOS OVULOS DATA ===")
congelamentos_ovulos_data = results['congelamentos_ovulos']
print(f"Records: {len(congelamentos_ovulos_data)}")
if not congelamentos_ovulos_data.empty:
    display(congelamentos_ovulos_data)
else:
    print("No congelamentos ovulos data found.")


=== CONGELAMENTOS OVULOS DATA ===
Records: 0
No congelamentos ovulos data found.


### 13. Descongelamentos Ovulos


In [225]:
print("=== DESCONGELAMENTOS OVULOS DATA ===")
descongelamentos_ovulos_data = results['descongelamentos_ovulos']
print(f"Records: {len(descongelamentos_ovulos_data)}")
if not descongelamentos_ovulos_data.empty:
    display(descongelamentos_ovulos_data)
else:
    print("No descongelamentos ovulos data found.")


=== DESCONGELAMENTOS OVULOS DATA ===
Records: 0
No descongelamentos ovulos data found.


### 14. Ovulos Congelados


In [226]:
print("=== OVULOS CONGELADOS DATA ===")
ovulos_congelados_data = results['ovulos_congelados']
print(f"Records: {len(ovulos_congelados_data)}")
if not ovulos_congelados_data.empty:
    display(ovulos_congelados_data)
else:
    print("No ovulos congelados data found.")


=== OVULOS CONGELADOS DATA ===
Records: 0
No ovulos congelados data found.


### 12. Planilha Embryoscope Combined


In [227]:
print("=== PLANILHA EMBRYOSCOPE COMBINED DATA ===")
combined_planilha_data = results['planilha_embryoscope_combined']
print(f"Records: {len(combined_planilha_data)}")
if not combined_planilha_data.empty:
    display(combined_planilha_data)
else:
    print("No planilha embryoscope combined data found.")


=== PLANILHA EMBRYOSCOPE COMBINED DATA ===
Records: 0
No planilha embryoscope combined data found.
